<a href="https://colab.research.google.com/github/Al-Jazzazi/Final_Project_ML/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("chethuhn/network-intrusion-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/chethuhn/network-intrusion-dataset/versions/1


In [2]:
all_files = [
    "Friday-WorkingHours-Morning.pcap_ISCX.csv",
    "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "Wednesday-workingHours.pcap_ISCX.csv",
    "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "Monday-WorkingHours.pcap_ISCX.csv",
    "Tuesday-WorkingHours.pcap_ISCX.csv",
    "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
]

In [3]:
import os
import pandas as pd
import numpy as np

# Combine all CSV files into a single dataframe
dataframes = []
for file in all_files:
    # Make sure to replace with the correct file path
    df = pd.read_csv(os.path.join(path, file))
    dataframes.append(df)

In [4]:
# Concatenate all dataframes
df = pd.concat(dataframes, ignore_index=True)

In [5]:
# Data cleaning: Replace infinity with NaN and drop rows with NaN values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [6]:
# Extract features and labels
X = np.array(df.drop(columns=' Label'))
y = np.array(df[' Label'])

In [7]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [8]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
from tensorflow.keras.utils import to_categorical

split_index = int(0.8 * len(X))

X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
Y_onehot = to_categorical(y, num_classes=15)
# Sequential split
X_train, X_test = X_reshaped[:split_index], X_reshaped[split_index:]
y_train, y_test = Y_onehot[:split_index], Y_onehot[split_index:]

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))  # LSTM layer
model.add(Dense(32, activation='relu'))  # Dense hidden layer
model.add(Dense(15, activation='softmax'))  # Output layer for 15 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 64)                  │          36,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │             495 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 39,183 (153.06 KB)

 Trainable params: 39,183 (153.06 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
70697/70697 ━━━━━━━━━━━━━━━━━━━━ 258s 4ms/step - accuracy: 0.9804 - loss: 0.0615 - val_accuracy: 0.4863 - val_loss: 21.1468
Epoch 2/20
70697/70697 ━━━━━━━━━━━━━━━━━━━━ 260s 4ms/step - accuracy: 0.9899 - loss: 0.0250 - val_accuracy: 0.4897 - val_loss: 21.7266
Epoch 3/20
70697/70697 ━━━━━━━━━━━━━━━━━━━━ 245s 3ms/step - accuracy: 0.9915 - loss: 0.0222 - val_accuracy: 0.4899 - val_loss: 22.0518
Epoch 4/20
70697/70697 ━━━━━━━━━━━━━━━━━━━━ 252s 3ms/step - accuracy: 0.9922 - loss: 0.0207 - val_accuracy: 0.4894 - val_loss: 23.1728
Epoch 5/20
48638/70697 ━━━━━━━━━━━━━━━━━━━━ 1:02 3ms/step - accuracy: 0.9929 - loss: 0.0195